In [1]:
import pandas as pd
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
#regex = re.compile('[%s]' % re.escape(string.punctuation))

#Preprocess function
def preprocess_text(text):
    # insert a space between hashtags
    text = re.sub("\#"," \#", text)
    
    # remove urls 
    text = re.sub("https\:\/\/.*\s","",text)
    
    # remove HTML tags
    text = re.sub("\<.*\>","",text)
    
    # remove punctuation
    text = text.translate(str.maketrans('', '', punctuation))
    
    # lemmatize and tokenize
    tokens = mystem.lemmatize(text.lower())
    
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " "]
    
    text = " ".join(tokens)
    
    return text.strip()


In [2]:
import csv

# read in CSV file; *NOTE: use python engine to avoid column count errors
df = pd.read_csv(r"C:\Users\572719\Documents\115 Data\circus.csv", skiprows=7, engine="python")

In [3]:
# drop any header rows that are not the header
df.drop(df.index[(df["No"] == "No") & (df["Source"] == "Source") & (df["Host"] == "Host")].tolist(), inplace=True)

# [Optional] drop "No" column, which contains indices for each media source
#df.drop("No",axis=1, inplace=True)

# reset index to account for those dropped rows
df.reset_index(drop=True, inplace=True)


In [ ]:
import re

# initialize list to hold all processed text prepared for analysis
processedtext = []

# get text from content field and process (tokenize/lemmitize) for anaylsis
# *NOTE: there are nan rows that need to be dropped, because some sources don't have content (e.g. Tumblr)
contents = df["Contents"].dropna()

for text in contents:
    processedtext.append(preprocess_text(text))


In [ ]:
# get summary text for those entries that don't have content data 
summaries = list(df[df["Source"] == "TUMBLR"]["Summary"].dropna())

# process summaries and add to processedtext
for text in summaries:
    processedtext.append(preprocess_text(text))

processedtext